In [1]:
import json
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import string
import soundfile as sf

CTC_VOCAB = [''] + list(string.ascii_lowercase + string.digits) + [' ']

In [3]:
def parse(serialized_example):

    data_fields = {
        'waveforms': tf.compat.v1.VarLenFeature(tf.float32),
        'targets': tf.compat.v1.VarLenFeature(tf.int64),
        'targets_length': tf.compat.v1.VarLenFeature(tf.int64),
        'lang': tf.compat.v1.VarLenFeature(tf.int64),
    }
    features = tf.compat.v1.parse_single_example(
        serialized_example, features=data_fields
    )
    for k in features.keys():
        features[k] = features[k].values

    keys = list(features.keys())
    for k in keys:
        if k not in ['waveforms', 'waveforms_length', 'targets']:
            features.pop(k, None)

    return features

In [4]:
# !wget https://raw.githubusercontent.com/huseinzol05/malaya-speech/master/data/mixed-stt/huggingface-3mixed-train-test.json

In [5]:
with open('huggingface-3mixed-train-test.json') as fopen:
    data = json.load(fopen)
    
data.keys()

dict_keys(['train', 'test'])

In [6]:
singlish = sorted([d for d in data['train'] if 'singlish/' in d])
len(singlish)

977

In [7]:
import requests

def download_file_cloud(url, filename):
    try:
        r = requests.get(url, stream=True)
        total_size = int(r.headers['content-length'])
        version = int(r.headers.get('X-Bz-Upload-Timestamp', 0))
        try:
            local_size = os.path.getsize(filename)
            if local_size == total_size:
                print(f'{filename} local size matched with cloud size')
                return version
        except Exception as e:
            print(e)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'wb') as f:
            for data in r.iter_content(chunk_size=1_048_576):
                f.write(data)
    except Exception as e:
        print(f'download_file_cloud error: {e}')

In [8]:
directory = 'tfrecord'
!mkdir {directory}

mkdir: cannot create directory ‘tfrecord’: File exists


In [9]:
!mkdir wav text done

mkdir: cannot create directory ‘wav’: File exists
mkdir: cannot create directory ‘text’: File exists
mkdir: cannot create directory ‘done’: File exists


In [10]:
# !rm done/*

In [ ]:
from tqdm import tqdm

num_cpu_threads=4
thread_count=12
for f in tqdm(singlish):
    filename = os.path.split(f)[1]
    tfrecord = os.path.join(directory, filename)
    done_tfrecord = f'done/{filename}'
    if os.path.exists(done_tfrecord):
        continue
        
    download_file_cloud(f, tfrecord)
    try:
        d = tf.data.Dataset.from_tensor_slices(tf.constant([tfrecord]))
        cycle_length = num_cpu_threads
        d = d.interleave(
            tf.data.TFRecordDataset,
            cycle_length=cycle_length,
            block_length=thread_count)
        d = d.map(parse, num_parallel_calls=num_cpu_threads)
        d = d.as_numpy_iterator()
        f = os.path.split(tfrecord)[1].replace('.', '-')
        count = 0
        while True:
            try:
                new_f = f'{f}-{count}'
                f_wav = os.path.join('wav', new_f + '.wav')
                f_text = os.path.join('text', new_f + '.txt')
                r = next(d)
                w = r['waveforms']
                text = ''.join([CTC_VOCAB[t] for t in r['targets']])

                sf.write(f_wav, w, 16000)
                with open(f_text, 'w') as fopen:
                    fopen.write(text)

                count += 1
            except Exception as e:
                print(e)
                break

        os.system(f'rm {tfrecord}')
        with open(done_tfrecord, 'w') as fopen:
            fopen.write('done')
    except Exception as e:
        print(e)

  0%|                                                   | 0/977 [00:00<?, ?it/s]2022-11-29 23:15:10.712291: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-29 23:15:10.712330: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: husein-MS-7D31
2022-11-29 23:15:10.712336: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: husein-MS-7D31
2022-11-29 23:15:10.712403: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-11-29 23:15:10.712427: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.141.3
2022-11-29 23:15:10.712431: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.141.3
2022-11-29 23:15:10.713074: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optim


[Errno 2] No such file or directory: 'tfrecord/1-113.tfrecord'


 14%|█████▉                                   | 141/977 [03:47<26:43,  1.92s/it]


[Errno 2] No such file or directory: 'tfrecord/1-114.tfrecord'


 15%|█████▉                                   | 142/977 [05:38<48:13,  3.47s/it]

...


[Errno 2] No such file or directory: 'tfrecord/7-56.tfrecord'


 95%|████████████████████████████████████▏ | 930/977 [15:25:32<27:26, 35.03s/it]


[Errno 2] No such file or directory: 'tfrecord/7-57.tfrecord'


 95%|████████████████████████████████████▏ | 931/977 [15:26:25<30:58, 40.41s/it]


[Errno 2] No such file or directory: 'tfrecord/7-58.tfrecord'
